In [1]:
import re
import numpy as np
import pandas as  pd
from pprint import pprint# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel# spaCy for preprocessing
import spacy# Plotting tools
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
import pickle

from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

#load pkl in a list
def load_corpus(pkl_path):
    rfc_dict = pickle.load(open( 'rfc_pkl.pkl', "rb" ))
    return list(rfc_dict.values())

# pkl is saved as a list of strings
cleaned_rfcs = load_corpus('rfc_pkl.pkl')
data_words = []
for s in cleaned_rfcs:
    data_words.append(s.split(' '))

In [2]:
bigram = gensim.models.phrases.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.phrases.Phrases(bigram[data_words], threshold=100)
# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)
# See trigram example

# Define function for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

/home/prakhar/anaconda3/envs/onr/lib/python3.5/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [3]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])
# print(type(data_lemmatized))
# print(data_lemmatized[0])

In [4]:
id2word = corpora.Dictionary(data_lemmatized)  
# Create Corpus 
texts = data_lemmatized  
# Term Document Frequency 
corpus = [id2word.doc2bow(text) for text in texts]  
# View 
print('corpus type: ', type(corpus))

corpus type:  <class 'list'>


In [5]:
num_topics = 50
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus, id2word=id2word, num_topics=num_topics, random_state=100, 
update_every=1, chunksize=100, passes=10, alpha='auto', per_word_topics=True)
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]
# coherence and perplexity : metrics for topic model

[(21,
  '0.043*"nfsv" + 0.035*"lock" + 0.032*"file" + 0.029*"oper" + 0.026*"lfb" + '
  '0.021*"filehandl" + 0.019*"case" + 0.018*"open" + 0.017*"stateid" + '
  '0.016*"compon"'),
 (13,
  '0.050*"locat" + 0.028*"inform" + 0.027*"element" + 0.027*"xml" + '
  '0.022*"xselement" + 0.021*"presenc" + 0.021*"datum" + 0.017*"devic" + '
  '0.013*"provid" + 0.013*"schema"'),
 (10,
  '0.073*"printer" + 0.038*"job" + 0.031*"ipp" + 0.027*"telnet" + '
  '0.019*"print" + 0.012*"log" + 0.011*"user" + 0.011*"rfc" + 0.010*"suppli" + '
  '0.010*"command"'),
 (46,
  '0.025*"mrb" + 0.022*"code" + 0.021*"decod" + 0.016*"xsdelement" + '
  '0.012*"broker" + 0.011*"hip" + 0.011*"puzzl" + 0.011*"track" + 0.010*"silk" '
  '+ 0.009*"return"'),
 (42,
  '0.071*"flow" + 0.034*"inform" + 0.033*"jw" + 0.027*"templat" + 0.027*"rfc" '
  '+ 0.025*"element" + 0.023*"select" + 0.021*"export" + 0.021*"record" + '
  '0.020*"process"'),
 (1,
  '0.026*"interv" + 0.020*"line" + 0.019*"current" + 0.019*"second" + '
  '0.017*"int

In [6]:
print('\nPerplexity: ', lda_model.log_perplexity(corpus))
coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)



Perplexity:  -6.954321521155356

Coherence Score:  0.44048501159229575


In [7]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/home/prakhar/anaconda3/envs/onr/lib/python3.5/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=           Freq  cluster  topics         x         y
topic                                               
22     9.792728        1       1 -0.051061 -0.088877
20     9.135039        1       2 -0.000319 -0.144572
18     5.841356        1       3 -0.052339  0.016058
7      5.823073        1       4 -0.038212 -0.116036
8      4.604327        1       5  0.001051 -0.117284
35     3.017412        1       6  0.171615 -0.138927
11     2.992961        1       7  0.175775 -0.072626
43     2.695724        1       8 -0.010277 -0.049211
19     2.617992        1       9 -0.082665 -0.038831
2      2.552317        1      10  0.002009 -0.067499
15     2.266625        1      11  0.020268 -0.099823
31     2.195655        1      12 -0.020867 -0.087351
41     2.190288        1      13 -0.162682 -0.062315
34     2.140875        1      14  0.107906 -0.059526
4      2.094959        1      15  0.143907 -0.026620
45     2.020875        1      16 -0.123201 -0.060111
23     2.002022        1      17  0.117051 -0.108325
44     1.765436        1      18  0.115961 -0.025835
6      1.731769        1      19  0.098642 -0.067823
48     1.654398        1      20 -0.107925 -0.049767
30     1.615378        1      21 -0.112165 -0.058051
3      1.570079        1      22  0.089177 -0.095234
47     1.516646        1      23 -0.181456  0.010421
37     1.499563        1      24  0.083027 -0.045952
5      1.495598        1      25  0.157358 -0.017924
49     1.448668        1      26 -0.047156  0.084403
0      1.439280        1      27 -0.124748 -0.083080
28     1.391603        1      28  0.068687  0.013972
14     1.361368        1      29 -0.138625 -0.016125
33     1.357246        1      30 -0.084296 -0.047600
40     1.233507        1      31  0.132907 -0.053830
17     1.155195        1      32  0.017538  0.011519
16     1.139690        1      33 -0.082499 -0.004565
24     1.138206        1      34  0.156835  0.008682
27     1.034849        1      35  0.100121  0.064266
9      0.958619        1      36 -0.131158  0.011696
32     0.956175        1      37 -0.065200  0.044589
36     0.925872        1      38 -0.046325  0.009554
12     0.869978        1      39 -0.173603  0.020215
1      0.768052        1      40  0.048350  0.124293
38     0.763283        1      41  0.098272  0.147471
13     0.742878        1      42 -0.113407  0.104836
26     0.739254        1      43  0.014214  0.091849
21     0.709582        1      44 -0.009141  0.187985
42     0.708705        1      45 -0.047352  0.014724
39     0.570967        1      46  0.070020  0.185792
10     0.530735        1      47 -0.073548  0.136178
29     0.459410        1      48 -0.015195  0.104032
25     0.417737        1      49  0.118917  0.280270
46     0.346055        1      50 -0.014186  0.230915, topic_info=       Category           Freq                      Term          Total  \
term                                                                      
87443   Default   58332.000000                    server   58332.000000   
9777    Default   47716.000000                    client   47716.000000   
11322   Default   34919.000000                  attribut   34919.000000   
129408  Default   83695.000000                   address   83695.000000   
107417  Default   94424.000000                    messag   94424.000000   
82822   Default   42757.000000                    header   42757.000000   
122763  Default   55117.000000                     field   55117.000000   
73457   Default   55609.000000                    object   55609.000000   
45089   Default   64312.000000                   request   64312.000000   
77744   Default   53792.000000                    packet   53792.000000   
52475   Default   39280.000000                       key   39280.000000   
57526   Default   95267.000000                      valu   95267.000000   
10153   Default   25909.000000                      rout   25909.000000   
39947   Default   28160.000000                   element   28160.000000   
8